In [1]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langsmith import Client
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini")
no_rag_chain = llm | StrOutputParser()
print("=== No RAG ===")
result1 = no_rag_chain.invoke("What is OpenClaw?")
print(result1)


=== No RAG ===
As of my last update in October 2023, there wasn't a widely recognized technology or concept known as "OpenClaw." It's possible that it could refer to a niche project, an open-source software initiative, or a specific tool or library developed after my last update. 

If you could provide more context or specific details about what "OpenClaw" pertains to, I may be able to help you more effectively! Alternatively, you might want to check the latest online resources or news for the most current information.


In [4]:
loader = UnstructuredMarkdownLoader("./openclaw_info.md")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, add_start_index=True)
all_splits = text_splitter.split_documents(docs)

vector_store = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    persist_directory="./chroma_embeddings"
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("\n=== RAG ===")
print("Prompt: " + prompt.messages[0].prompt.template + "\n")
questions = ["Explain OpenClaw in detail.", "How to install OpenClaw?", "How does OpenClaw work?"]
for question in questions:
    print(f"Question: {question}")
    result2 = rag_chain.invoke(question)
    print(result2 + "\n")


=== RAG ===
Prompt: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:

Question: Explain OpenClaw in detail.
OpenClaw is a system designed specifically for Molty, an AI assistant modeled as a space lobster. However, the context does not provide detailed information about OpenClaw's features or functionalities. Therefore, I'm unable to provide further details.

Question: How to install OpenClaw?
To install OpenClaw, first clone the repository using the command `git clone https://github.com/openclaw/openclaw.git`, then navigate into the directory with `cd openclaw`.

Question: How does OpenClaw work?
OpenClaw is a personal AI assistant designed to operate on your own devices. Its functionality allows for personalized assistance while ensuring user priva

In [5]:
another_prompt = client.pull_prompt("ahmedghani/agentic-rag")
another_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | another_prompt
    | llm
    | StrOutputParser()
)
print("\n=== Another RAG Prompt ===")
print("Prompt: " + another_prompt.messages[0].prompt.template + "\n")
for question in questions:
    print(f"Question: {question}")
    result3 = another_rag_chain.invoke(question)
    print(result3+ "\n")


=== Another RAG Prompt ===
Prompt: You are an advanced assistant designed for question-answering and task execution using retrieved context. Use the provided pieces of context to accurately answer the question or perform the tasks requested by the user. If the context is insufficient, acknowledge it. Keep your responses concise and to the point.
Context: {context}
Question: {question}
Answer:

Question: Explain OpenClaw in detail.
OpenClaw is an AI system specifically designed for Molty, who serves as a space lobster assistant. The context does not provide further details on its functionalities, capabilities, or design specifications beyond its purpose to assist Molty.

Question: How to install OpenClaw?
The provided context does not include specific installation instructions for OpenClaw beyond cloning the repository. To install it, you may need to refer to the project's documentation or README file in the cloned repository for detailed installation steps. You can also check for any 